In [ ]:
%run ../../cpch_source.py
import os
import time

In [ ]:
np.random.seed(211)
#run script total of 3 times, for r=m=2, m=3, r=2, and r=m=3, in that order
m = 3
r = 2
N = 10000
power_iters = 100
total_iters = 100
mu_max = 4
mu_step = 0.2
alpha = 0.05

In [ ]:
mu_grid = np.arange(0, mu_max + mu_step, mu_step)
def null_mu(m, r, mu_grid):
    '''
    generates list of n-length arrays of unique configurations under the null using mu_grid
    '''
    all_prods = list(itertools.product(mu_grid, repeat = m))
    null_cases = np.unique(np.array([sorted(i) for i in all_prods if sum(np.array(i) == 0.0) in np.arange(m-r+1, m+1, 1)]), axis = 0)
    return(null_cases)

In [ ]:
mu_grid = np.arange(0, mu_max + mu_step, mu_step)
null_cases = null_mu(m, r, mu_grid)
num_null_cases = len(null_cases)
print(num_null_cases)
def calculate_cpch(null_case, power_iters, m, r):
    #for a given mu vector, generate data from that mu vector
    XX = norm.rvs(null_case, scale = 1, size = (power_iters, m))
    if m == r:
        cpch_f = cpch(XX, m, r, f_fisher, norm.pdf, norm.cdf, truncnorm.rvs, N) 
        return (cpch_f, cpch_f, cpch_f)
    else:
        cpch_f = cpch(XX, m, r, f_fisher, norm.pdf, norm.cdf, truncnorm.rvs, N) 
        cpch_b = cpch(XX, m, r, f_bonferroni, norm.pdf, norm.cdf, truncnorm.rvs, N)
        cpch_s = cpch(XX, m, r, f_simes, norm.pdf, norm.cdf, truncnorm.rvs, N) 
        return (cpch_f, cpch_s, cpch_b)

In [ ]:
#save a csv for each null case
for null_case in null_cases:
    #change path to where you want data to be saved
    path = 'Data/validity_analysis/m_' + str(m) + '_r_' + str(r) + '_mu_' + str(list(np.round(null_case, 3))).replace(', ', '_').replace('[', '').replace(']', '')
    isExist = os.path.exists(path)
    if not isExist:
        os.mkdir(path) 
    start = time.time()
    for iteration in np.arange(1, total_iters+1, 1):
        np.random.seed(int(iteration * 1000 + m * 100 + r))
        config =  'm=' + str(m) + ' , r=' + str(r) + ', locs=' + str(list(np.round(null_case, 3)))
        cpch_f, cpch_b, cpch_s = calculate_cpch(null_case, power_iters, m, r)
        df = pd.DataFrame({'iteration': np.repeat(iteration, power_iters)
                  , 'm': np.repeat(m, power_iters)
                  , 'r': np.repeat(r, power_iters)
                  , 'mu': np.repeat(str(list(np.round(null_case, 3))), power_iters)
                  , 'Configuration': np.repeat(config, power_iters)
                  , "cpch_f": cpch_f
                  , "cpch_s": cpch_s
                  , "cpch_b": cpch_b})
        df.to_csv(path + '/it_' + str(iteration) + '.csv', index = False)
    end = time.time()
    print ("Null Case " + str(null_case) + " Completed, Time elapsed:", (end - start)/60)

In [ ]:
#write script to knit together csvs across all iterations for each case, so will have config, and p-values
ks_f, ks_b, ks_s = [], [], []
for null_case in null_cases:
    path = 'Data/validity_analysis/m_' + str(m) + '_r_' + str(r) + '_mu_' + str(list(np.round(null_case, 3))).replace(', ', '_').replace('[', '').replace(']', '')
    li = []
    for filename in os.listdir(path):
        df = pd.read_csv(path + '/' + filename, index_col = None, header=0)
        li.append(df)
    df_final = pd.concat(li, axis = 0, ignore_index = True)
    df_final.to_csv('Data/final_data/m_' + str(m) + '_r_' + str(r) + '_mu_' + str(list(np.round(null_case, 3))).replace(', ', '_').replace('[', '').replace(']', '') + '.csv',
                   index = False)
    #get ks test statistic for each null case
    ks_f.append(scipy.stats.kstest(df_final['cpch_f'], 'uniform')[0])
    ks_s.append(scipy.stats.kstest(df_final['cpch_s'], 'uniform')[0])
    ks_b.append(scipy.stats.kstest(df_final['cpch_b'], 'uniform')[0])

In [ ]:
#make dataset that combines all p-values for all null configurations for a given m and r
combined_data = []
#change path to where you want data to be saved
path = 'Data/final_data'
for filename in os.listdir(path):
    if not filename.startswith('.'):
        df = pd.read_csv(path + '/' + filename, index_col = None, header=0)
        combined_data.append(df)
df_combined = pd.concat(combined_data, axis = 0, ignore_index = True).sort_values(['Configuration', 'iteration'])
#change path to where you want data to be saved
df_combined.to_csv('Data/combined_data_m_' + str(m) + '_r_' + str(r) + '.csv',
                index = False)

In [ ]:
#get ks statistics for a given m and r
ks_data = pd.DataFrame({'ks_f': ks_f, 'ks_b': ks_b, 'ks_s': ks_s})
ks_data.insert(0, 'm', m)
ks_data.insert(1, 'r', r)
ks_data.insert(2, 'mu', [str(list(np.round(null_case, 3))) for null_case in null_cases])
#ks_data['null_case'] = null_cases
config = ['m=' + str(m) + ' , r=' + str(r) + ', locs=' +str(list(np.round(null_case, 3))) for null_case in null_cases]
ks_data.insert(3, 'Configuration', config)
#change path to where you want data to be saved
ks_data.to_csv('Data/ks_df_m_' + str(m) + '_r_' + str(r) + '.csv', index = False)